In [25]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from test_functions import problem_setup
from sim_helpers import (
    gen_initial_real_data,
    fit_outcome_model,
    gen_random_candidates,
    fit_pref_model,
    gen_rand_X,
    gen_rand_points,
    PosteriorMeanDummySampler,
    gen_comps,
)
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
problem_strs_dict = {
    'dtlz2_6d4d_restrictednegsixhump': '6d-4d DTLZ2 (4d outputs) \n restricted negative six hump sum',
    'dtlz2_9d8d_restrictednegsixhump': '9d-8d DTLZ2 (8d outputs) \n restricted negative six hump sum',
    'dtlz2_6d4d_rotatedrestrictednegsixhump': '6d-4d DTLZ2 (4d outputs) \n rotated restricted neg. six hump sum',
    'dtlz2_9d8d_rotatedrestrictednegsixhump': '9d-8d DTLZ2 (8d outputs) \n rotated restricted neg. six hump sum',

    'gpdraw_4d6d_kumaraswamy': '4d-6d GP Draw (6d outputs) \n Kumaraswamy CDFs sum w/ interaction',
    'gpdraw_4d6d_piecewiselinear': '4d-6d GP Draw (6d outputs) \n piece-wise linear',

    "dtlz2_6d4d_piecewise": "6d-4d DTLZ2 (4d outputs) \n quartic-linear piece-wise",
    "dtlz2_6d4d_kumaraswamy": "6d-4d DTLZ2 (4d outputs) \n Kumaraswamy CDFs sum w/ interaction",
    "dtlz2_9d8d_kumaraswamy": "9d-8d DTLZ2 (8d outputs) \n Kumaraswamy CDFs sum w/ interaction",
    "dtlz2_9d8d_piecewise": "9d-8d DTLZ2 (8d outputs) \n quartic-linear piece-wise",
    
    'dtlz2_6d4d_negsixhump': '6d-4d DTLZ2 (4d outputs) \n negative six hump sum',
    'dtlz2_9d8d_negsixhump': '9d-8d DTLZ2 (8d outputs) \n negative six hump sum',
    'dtlz2_6d4d_quadraticsum': '6d-4d DTLZ2 (4d outputs) \n quadratic functions sum',    
    'dtlz2_9d8d_quadraticsum': '9d-8d DTLZ2 (8d outputs) \n quadratic functions sum',   

    'augdtlz2_4d8d_negl1dist': 'Augmented DTLZ2 (d=4, k=8) \n Negative L1 distance',
    'augdtlz2_8d8d_negl1dist': 'Augmented DTLZ2 (d=8, k=8) \n Negative L1 distance',
    'augdtlz2_8d12d_negl1dist': 'Augmented DTLZ2 (d=8, k=12) \n Negative L1 distance',
    'dtlz2_8d4d_negl1dist': 'DTLZ2 (d=8, k=4) \n Negative L1 distance',
    'dtlz2_8d4d_negl2dist': 'DTLZ2 (d=8, k=4) \n Negative L2 distance',
    'augdtlz2_8d16d_negl2dist': 'Augmented DTLZ2 (d=8, k=16) \n Negative L2 distance',
    'augdtlz2_8d16d_negl1dist': 'Augmented DTLZ2 (d=8, k=16) \n Negative L1 distance',

    "dtlz2_8d4d_idealnegl1dist": 'DTLZ2 (d=8, k=4) \n L1 distance from desideratum',
    "augdtlz2_8d16d_idealnegl1dist": 'Augmented DTLZ2 (d=8, k=16) \n L1 distance from desideratum',
    "dtlz2_8d4d_idealnegl2dist": 'DTLZ2 (d=8, k=4) \n L2 distance from desideratum',
    "augdtlz2_8d16d_idealnegl2dist": 'Augmented DTLZ2 (d=8, k=16) \n L2 distance from desideratum',
    
    "dtlz2_8d4d_idealnegl1dist": 'DTLZ2 (d=8, k=4) \n L1 distance from desideratum',
    "augdtlz2_8d16d_idealnegl1dist": 'Augmented DTLZ2 (d=8, k=16) \n L1 distance from desideratum',
    "dtlz2_8d4d_idealnegl2dist": 'DTLZ2 (d=8, k=4) \n L2 distance from desideratum',
    "augdtlz2_8d16d_idealnegl2dist": 'Augmented DTLZ2 (d=8, k=16) \n L2 distance from desideratum',

    "carcabdesign_7d9d_piecewiselinear": "Car cab design (d=7, k=9) \n piece-wise linear", 
    "vehiclesafety_5d3d_kumaraswamyproduct": "Vehicle safety (d=5, k=3) \n product of Kumaraswamy CDFs",
    'osy_6d8d_piecewiselinear': 'OSY (d=6, k=8) \n piece-wise linear',
    'osy_6d8d_kumaraswamy': 'OSY (d=6, k=8) \n Kumaraswamy CDFs sum w/ interaction',
    "carcabdesign_7d9d_linear": "Car cab design (d=7, k=9) \n linear",
    "vehiclesafety_5d3d_linear": "Vehicle safety (d=5, k=3) \n linear",
}

In [29]:
n_init_X_batch = 256
fixed_init_X_dict = {}

for problem_str in problem_strs_dict.keys():
    problem_prefix = "_".join(problem_str.split("_")[:2])
    (
        X_dim,
        Y_dim,
        problem,
        util_type,
        get_util,
        Y_bounds,
        probit_noise,
    ) = problem_setup(problem_str, noisy=False, dtype=torch.float64)
    
    if X_dim <= 5:
        init_n_outcome = 16
    else:
        init_n_outcome = 32
        
    if problem_prefix not in fixed_init_X_dict:
        fixed_init_X_dict[problem_prefix] = []
        for i in range(n_init_X_batch):
            fixed_init_X_dict[problem_prefix].append(gen_rand_X(init_n_outcome, problem))

dtlz2_6d4d_restrictednegsixhump, noisy: False, noise_std: 0
dtlz2_9d8d_restrictednegsixhump, noisy: False, noise_std: 0
dtlz2_6d4d_rotatedrestrictednegsixhump, noisy: False, noise_std: 0
dtlz2_9d8d_rotatedrestrictednegsixhump, noisy: False, noise_std: 0
gpdraw_4d6d_kumaraswamy, noisy: False, noise_std: 0
gpdraw_4d6d_piecewiselinear, noisy: False, noise_std: 0
dtlz2_6d4d_piecewise, noisy: False, noise_std: 0
dtlz2_6d4d_kumaraswamy, noisy: False, noise_std: 0
dtlz2_9d8d_kumaraswamy, noisy: False, noise_std: 0
dtlz2_9d8d_piecewise, noisy: False, noise_std: 0
dtlz2_6d4d_negsixhump, noisy: False, noise_std: 0
dtlz2_9d8d_negsixhump, noisy: False, noise_std: 0
dtlz2_6d4d_quadraticsum, noisy: False, noise_std: 0
dtlz2_9d8d_quadraticsum, noisy: False, noise_std: 0
augdtlz2_4d8d_negl1dist, noisy: False, noise_std: 0
augdtlz2_8d8d_negl1dist, noisy: False, noise_std: 0
augdtlz2_8d12d_negl1dist, noisy: False, noise_std: 0
dtlz2_8d4d_negl1dist, noisy: False, noise_std: 0
dtlz2_8d4d_negl2dist, noisy:

In [30]:
# pickle.dump(fixed_init_X_dict, open("../fixed_init_X_dict.pickle", "wb"))

In [33]:
for i in range(10):
    print(f"./run_multi_sim.sh -g{3+i%5} -b{i+1} -e{i+1} -cconstant")

./run_multi_sim.sh -g3 -b1 -e1 -cconstant
./run_multi_sim.sh -g4 -b2 -e2 -cconstant
./run_multi_sim.sh -g5 -b3 -e3 -cconstant
./run_multi_sim.sh -g6 -b4 -e4 -cconstant
./run_multi_sim.sh -g7 -b5 -e5 -cconstant
./run_multi_sim.sh -g3 -b6 -e6 -cconstant
./run_multi_sim.sh -g4 -b7 -e7 -cconstant
./run_multi_sim.sh -g5 -b8 -e8 -cconstant
./run_multi_sim.sh -g6 -b9 -e9 -cconstant
./run_multi_sim.sh -g7 -b10 -e10 -cconstant
